In [1]:
import pandas as pd
import numpy as np

In [2]:
df_amazon_sr = pd.read_csv(
    '../data/raw_csv_files/Amazon Sale Report.csv', low_memory=False)

In [3]:
df_amazon_sr.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [4]:
# Format columns
df_amazon_sr.columns = (
    df_amazon_sr.columns
    .str.strip()                                # Remove leading/trailing whitespace
    .str.lower()                                # Convert to lowercase
    .str.replace(r'[^\w\s]', '', regex=True)    # Remove special characters like - or .
    .str.replace(r'\s+', '_', regex=True)       # Replace spaces with underscores
)

In [5]:
df_amazon_sr.nunique()

index               128975
order_id            120378
date                    91
status                  13
fulfilment               2
sales_channel            2
shipservicelevel         2
style                 1377
sku                   7195
category                 9
size                    11
asin                  7190
courier_status           3
qty                     10
currency                 1
amount                1410
shipcity              8955
shipstate               69
shippostalcode        9459
shipcountry              1
promotionids          5787
b2b                      2
fulfilledby              1
unnamed_22               1
dtype: int64

In [6]:
# Drop unnecessary columns
cols_to_drop = [
    'index',
    'asin',
    'shipcountry',
    'unnamed_22'
]
df_amazon_sr.drop(columns=cols_to_drop, inplace=True)

In [7]:
df_amazon_sr['sales_channel'].value_counts()

sales_channel
Amazon.in     128851
Non-Amazon       124
Name: count, dtype: int64

In [8]:
# Drop 124 rows from non-Amazon sales_channel (all unshipped, not relevant)
df_amazon_sr = df_amazon_sr[df_amazon_sr['sales_channel'] == 'Amazon.in']

In [9]:
# Drop 'sales_channel' now that it's uniform

# Filter and make a safe copy
df_amazon_sr = df_amazon_sr[df_amazon_sr['sales_channel'] == 'Amazon.in'].copy()

# Now it's safe to drop the column
df_amazon_sr.drop(columns=['sales_channel'], inplace=True)

In [10]:
df_amazon_sr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128851 entries, 0 to 128974
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          128851 non-null  object 
 1   date              128851 non-null  object 
 2   status            128851 non-null  object 
 3   fulfilment        128851 non-null  object 
 4   shipservicelevel  128851 non-null  object 
 5   style             128851 non-null  object 
 6   sku               128851 non-null  object 
 7   category          128851 non-null  object 
 8   size              128851 non-null  object 
 9   courier_status    121979 non-null  object 
 10  qty               128851 non-null  int64  
 11  currency          121180 non-null  object 
 12  amount            121180 non-null  float64
 13  shipcity          128818 non-null  object 
 14  shipstate         128818 non-null  object 
 15  shippostalcode    128818 non-null  float64
 16  promotionids      79822 n

In [11]:
# Convert some data types

In [12]:
# Convert date to datetime64
df_amazon_sr['date'] = pd.to_datetime(
    df_amazon_sr['date'],
    format='%m-%d-%y',     # Specify expected format
    errors='coerce'        # Handle bad entries safely
)
print(df_amazon_sr['date'].dtypes)

datetime64[ns]


In [13]:
# Convert postal codes to strings so we don't lose any formatting (like leading zeros)
df_amazon_sr['shippostalcode'] = (
    df_amazon_sr['shippostalcode']
    .astype('Int64')   # keep nulls intact
    .astype(str)
)

In [14]:
# Confirm
print(df_amazon_sr[['date', 'shippostalcode']].dtypes)

date              datetime64[ns]
shippostalcode            object
dtype: object


In [15]:
# Show date rage of data
print('Date Range:')
print('Min:', df_amazon_sr['date'].min())
print('Max:', df_amazon_sr['date'].max())

Date Range:
Min: 2022-03-31 00:00:00
Max: 2022-06-29 00:00:00


In [16]:
# In India, Q1 FY23 = April 1, 2022 to June 30, 2022
# The original data starts on March 31 and ends on June 29 — likely due to time zone differences or US-based reporting
# Shift all dates forward by one day to align cleanly with Indian fiscal Q1 FY23

In [17]:
df_amazon_sr['date'] = df_amazon_sr['date'] + pd.Timedelta(days=1)

In [18]:
# Show date rage of data
print('Date Range:')
print('Min:', df_amazon_sr['date'].min())
print('Max:', df_amazon_sr['date'].max())

Date Range:
Min: 2022-04-01 00:00:00
Max: 2022-06-30 00:00:00


In [19]:
# Order the dataframe by date
df_amazon_sr = df_amazon_sr.sort_values(by='date').reset_index(drop=True)

In [20]:
df_amazon_sr.describe()

,date,qty,amount
count,128851,128851.000000,121180.000000
mean,2022-05-13 11:51:46.145858304,0.904005,648.561465
min,2022-04-01 00:00:00,0.000000,0.000000
25%,2022-04-21 00:00:00,1.000000,449.000000
50%,2022-05-11 00:00:00,1.000000,605.000000
75%,2022-06-05 00:00:00,1.000000,788.000000
max,2022-06-30 00:00:00,8.000000,5584.000000
std,NaN,0.308131,281.211687


In [21]:
# Rename some columns for claity
df_amazon_sr.rename(columns={
    'shipcity': 'city',
    'shipstate': 'state',
    'shippostalcode': 'postal_code',
    'shipservicelevel': 'shipping_method',
    'promotionids': 'promo_id',
    'fulfilledby': 'fulfilled_by'
}, inplace=True)

In [22]:
# Format Title Case for category values (consistency) & remove any white spaces
df_amazon_sr['category'] = df_amazon_sr['category'].str.strip().str.title()

In [23]:
# Re-eheck unique promo_ids
df_amazon_sr['promo_id'].nunique()

5787

In [24]:
# Looks very messy
df_amazon_sr['promo_id'].dropna().unique()[:10]

array(['Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-SL3Y2YRMAUNFC,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAVTYR4X7C,Amazon PLCC Free-Financing Universal Merchant AAT-XXRCW6NZEPZI4,Amazon PLCC Free-Financing Universal Merchant AAT-WSJLDN3X7KEMO,Amazon PLCC Free-Financing Universal Merchant AAT-VL6FGQVGQVXUS,Amazon PLCC Free-Financing Universal Merchant AAT-EOKPWFWYW7Y6I,Amazon PLCC Free-Financing Universal Merchant AAT-ZYL5UPUNW6T62,Amazon PLCC Free-Financing Universal Merchant AAT-G5ZRX4BZOIODU,Amazon PLCC Free-Financing Universal Merchant AAT-ACZIPIA37BFIY,Amazon PLCC Free-Financing Universal Merchant AAT-YFJA6XMAMWZSK,Amazon PLCC Free-Financing Universal Merchant AAT-K66BSCE2J2V5E,Amazon PLCC Free-Financing Universal Merchant AAT-DFM4OAAQ3RGPM,Amazon PLCC Free-Financing Unive

In [25]:
# Cut out everything after 'Free-Financing' (it's meaningless) and assign to new column
df_amazon_sr['promo_group'] = df_amazon_sr['promo_id'].str.extract(r'^(.*?Free-Financing)')

In [26]:
# Fill in all NaNs in promo_group with 'None'
df_amazon_sr['promo_group'] = df_amazon_sr['promo_group'].fillna('None')

In [27]:
# Check value counts
df_amazon_sr['promo_group'].value_counts()

promo_group
None                          96503
Amazon PLCC Free-Financing    32348
Name: count, dtype: int64

In [28]:
# Check number of uniques 
df_amazon_sr['promo_group'].nunique()
# 5787 -> 2

2

In [29]:
# Drop the original, messy 'promo_id' column
df_amazon_sr.drop(columns=['promo_id'], inplace=True)

In [30]:
# Get original column name back; rename 'promo_group' to 'promo_id'
df_amazon_sr.rename(columns={'promo_group': 'promo_id'}, inplace=True)

In [31]:
# Fill in all NaNs in fulfilled_by with 'Other'
df_amazon_sr['fulfilled_by'] = df_amazon_sr['fulfilled_by'].fillna('Other')

In [32]:
# Clean up 'city' & 'state' columns now

In [33]:
# First, format city & state columns to have same Title Case, strip away white space
df_amazon_sr['city'] = df_amazon_sr['city'].str.strip().str.title()
df_amazon_sr['state'] = df_amazon_sr['state'].str.strip().str.title()

In [34]:
# Start with 'state'
df_amazon_sr['state'].nunique()

47

In [35]:
# View list of unique values
sorted(df_amazon_sr['state'].dropna().unique().tolist())

['Andaman & Nicobar',
 'Andhra Pradesh',
 'Apo',
 'Ar',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra And Nagar',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'New Delhi',
 'Nl',
 'Odisha',
 'Orissa',
 'Pb',
 'Pondicherry',
 'Puducherry',
 'Punjab',
 'Punjab/Mohali/Zirakpur',
 'Rajasthan',
 'Rajshthan',
 'Rajsthan',
 'Rj',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [36]:
# Standardize common spelling errors and abbreviations in 'state'
state_corrections = {
    'Rajshthan': 'Rajasthan',
    'Rajsthan': 'Rajasthan',
    'Rj': 'Rajasthan',
    'Orissa': 'Odisha',                                             # Official state name
    'Pondicherry': 'Puducherry',
    'Pb': 'Punjab',
    'Punjab/Mohali/Zirakpur': 'Punjab',                             # Keep only state
    'New Delhi': 'Delhi',                                           # Normalize NCR variant
    'Nl': 'Nagaland',
    'Ar': 'Arunachal Pradesh',
    'Apo': None,                                                    # Likely invalid
    'Dadra And Nagar': 'Dadra And Nagar Haveli And Daman And Diu'   # Official UT name
}
df_amazon_sr['state'] = df_amazon_sr['state'].replace(state_corrections)

In [37]:
df_amazon_sr['state'].nunique()

36

In [38]:
df_amazon_sr['state'].isnull().sum()

np.int64(34)

In [39]:
df_amazon_sr[df_amazon_sr['state'].isnull()]

,order_id,date,status,fulfilment,shipping_method,style,sku,category,size,courier_status,qty,currency,amount,city,state,postal_code,b2b,fulfilled_by,promo_id
4564,405-0034289-0259545,2022-04-04,Cancelled,Amazon,Expedited,J0077,J0077-SKD-S,Set,S,Cancelled,0,NaN,NaN,Apo,None,959121,False,Other,None
12181,407-3064376-9158743,2022-04-09,Cancelled,Merchant,Standard,JNE3405,JNE3405-KR-S,Kurta,S,NaN,0,INR,380.0,NaN,NaN,<NA>,False,Easy Ship,None
12182,407-3064376-9158743,2022-04-09,Cancelled,Merchant,Standard,JNE3567,JNE3567-KR-M,Kurta,M,NaN,0,INR,380.0,NaN,NaN,<NA>,False,Easy Ship,None
17721,404-7506843-7913132,2022-04-13,Shipped - Delivered to Buyer,Merchant,Standard,SET350,SET350-KR-NP-L,Set,L,Shipped,1,INR,1299.0,NaN,NaN,<NA>,False,Easy Ship,Amazon PLCC Free-Financing
23410,406-4079063-8291520,2022-04-16,Cancelled,Amazon,Expedited,JNE3567,JNE3567-KR-XXL,Kurta,XXL,Unshipped,1,INR,399.0,NaN,NaN,<NA>,False,Other,None
24952,402-8628677-0457954,2022-04-17,Shipped - Returned to Seller,Merchant,Standard,J0003,J0003-SET-S,Set,S,Shipped,1,INR,654.0,NaN,NaN,<NA>,False,Easy Ship,Amazon PLCC Free-Financing
29561,405-4927647-8064368,2022-04-20,Shipped,Amazon,Expedited,J0230,J0230-SKD-XS,Set,XS,Shipped,1,INR,1112.0,NaN,NaN,<NA>,False,Other,None
31484,402-4919636-4333150,2022-04-21,Shipped - Delivered to Buyer,Merchant,Standard,JNE1233,JNE1233-BLUE-KR-031-XXXL,Kurta,3XL,Shipped,1,INR,376.0,NaN,NaN,<NA>,False,Easy Ship,Amazon PLCC Free-Financing
33945,404-9229894-8608305,2022-04-22,Shipped,Amazon,Expedited,JNE3510,JNE3510-KR-M,Kurta,M,Shipped,1,INR,442.0,NaN,NaN,<NA>,False,Other,None
34774,407-4532637-8415521,2022-04-23,Cancelled,Merchant,Standard,JNE3405,JNE3405-KR-S,Kurta,S,NaN,0,INR,380.0,NaN,NaN,<NA>,False,Easy Ship,None


In [40]:
# Dropped 34 rows missing both 'city' and 'state' — can't be used for location analysis
df_amazon_sr = df_amazon_sr[~(df_amazon_sr['state'].isnull() & df_amazon_sr['city'].isnull())]

In [41]:
# Check
sorted(df_amazon_sr['state'].dropna().unique().tolist())

['Andaman & Nicobar',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra And Nagar Haveli And Daman And Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [42]:
# Final Check
sorted(df_amazon_sr['state'].dropna().unique().tolist())

['Andaman & Nicobar',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra And Nagar Haveli And Daman And Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [43]:
# Now 'city'
df_amazon_sr['city'].nunique()

7297

In [44]:
df_amazon_sr['postal_code'].nunique()

9459

In [45]:
lookup_postal = (
    df_amazon_sr[['postal_code', 'state', 'city']]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [46]:
lookup_postal.shape

(14436, 3)

In [47]:
lookup_postal.head()

,postal_code,state,city
0,700124,West Bengal,Kolkata
1,560090,Karnataka,Bengaluru
2,122001,Haryana,Gurgaon
3,401209,Maharashtra,Nalasopara East
4,500024,Telangana,Hyderabad


In [48]:
# Read the file (GeoNames, tab-separated, no header)
geo_postal = pd.read_csv(
    '../data/geonames_india.txt',   # ← update this path
    sep='\t',
    header=None,
    dtype={1: str}  # ensure postal_code keeps leading zeros
)

In [49]:
geo_postal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,IN,744301,Sawai,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,7.5166,93.6031,4
1,IN,744301,Kakana,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1167,92.8000,4
2,IN,744301,Mus,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.2333,92.7833,4
3,IN,744301,Lapathy,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
4,IN,744301,Carnicobar,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3


In [50]:
# Assign column names (from the GeoNames readme.txt file)
geo_postal.columns = [
    'country_code', 'postal_code', 'place_name', 'admin_name1', 'admin_code1',
    'admin_name2', 'admin_code2', 'admin_name3', 'admin_code3',
    'latitude', 'longitude', 'accuracy'
]

In [51]:
geo_postal.head()

,country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy
0,IN,744301,Sawai,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,7.5166,93.6031,4
1,IN,744301,Kakana,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1167,92.8000,4
2,IN,744301,Mus,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.2333,92.7833,4
3,IN,744301,Lapathy,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
4,IN,744301,Carnicobar,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3


In [52]:
geo_postal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155570 entries, 0 to 155569
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   country_code  155570 non-null  object 
 1   postal_code   155570 non-null  object 
 2   place_name    155569 non-null  object 
 3   admin_name1   155570 non-null  object 
 4   admin_code1   155570 non-null  int64  
 5   admin_name2   155570 non-null  object 
 6   admin_code2   153257 non-null  float64
 7   admin_name3   147534 non-null  object 
 8   admin_code3   0 non-null       float64
 9   latitude      155570 non-null  float64
 10  longitude     155570 non-null  float64
 11  accuracy      155570 non-null  int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 14.2+ MB


In [53]:
# Filter only rows with valid lat/lon and place_name
geo_filtered = geo_postal[
    geo_postal['place_name'].notnull() &
    geo_postal['latitude'].notnull() &
    geo_postal['longitude'].notnull()
].copy()

# Keep best (lowest) accuracy row for each (postal_code, admin_name1)
geo_grouped = (
    geo_filtered.sort_values(by='accuracy')
    .groupby(['postal_code', 'admin_name1'], as_index=False)
    .first()
)
# Rename columns for merge compatibility
geo_grouped.rename(columns={
    'place_name': 'city_geo',
    'admin_name1': 'state'
}, inplace=True)

In [54]:
# Let's see if GeoNames format of the state values is = to ours
set(geo_grouped['state'].unique()) == set(df_amazon_sr['state'].unique())

False

In [55]:
set(df_amazon_sr['state'].unique()) - set(geo_grouped['state'].unique())


{'Andaman & Nicobar',
 'Chhattisgarh',
 'Dadra And Nagar Haveli And Daman And Diu',
 'Ladakh',
 None,
 'Puducherry'}

In [56]:
# Match Amazon state names to GeoNames format
amazon_state_corrections = {
    'Andaman & Nicobar': 'Andaman & Nicobar Islands',
    'Chhattisgarh': 'Chattisgarh',
    'Dadra And Nagar Haveli And Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Ladakh': 'Jammu & Kashmir',
    'Puducherry': 'Pondicherry'
}
df_amazon_sr['state'] = df_amazon_sr['state'].replace(amazon_state_corrections)

In [57]:
# Merge enriched city and coordinates into main DataFrame
df_amazon_sr = df_amazon_sr.merge(
    geo_grouped[['postal_code', 'state', 'city_geo', 'latitude', 'longitude']],
    on=['postal_code', 'state'],
    how='left'
)

In [58]:
# Let's see if GeoNames format of the state values is now = to ours
set(geo_grouped['state'].unique()) == set(df_amazon_sr['state'].unique())

False

In [59]:
# It checks out, only value is 'None' which geo_grouped does not have
set(df_amazon_sr['state'].unique()) - set(geo_grouped['state'].unique())

{None}

In [60]:
# Compare cleaned 'city' to enriched 'city_geo'
df_amazon_sr['city_match'] = (
    df_amazon_sr['city'] == df_amazon_sr['city_geo']
)

# Check match counts
df_amazon_sr['city_match'].value_counts()


city_match
False    128118
True        700
Name: count, dtype: int64

In [61]:
df_amazon_sr.head()

,order_id,date,status,fulfilment,shipping_method,style,sku,category,size,courier_status,...,city,state,postal_code,b2b,fulfilled_by,promo_id,city_geo,latitude,longitude,city_match
0,404-1445673-1345134,2022-04-01,Shipped - Delivered to Buyer,Merchant,Standard,JNE3724,JNE3724-KR-L,Kurta,L,Shipped,...,Kolkata,West Bengal,700124,False,Easy Ship,Amazon PLCC Free-Financing,Barasat H.O,22.4656,88.7803,False
1,406-7387241-2881923,2022-04-01,Shipped,Amazon,Expedited,SET187,SET187-KR-DH-XL,Set,XL,Shipped,...,Bengaluru,Karnataka,560090,False,Other,None,Chikkabanavara,13.2257,77.5750,False
2,402-9332104-0945115,2022-04-01,Cancelled,Merchant,Standard,JNE3383,JNE3383-KR-A-M,Kurta,M,NaN,...,Gurgaon,Haryana,122001,False,Easy Ship,None,Basai Road,28.4180,76.9839,False
3,408-1008226-2450750,2022-04-01,Shipped,Amazon,Expedited,JNE3405,JNE3405-KR-L,Kurta,L,Shipped,...,Nalasopara East,Maharashtra,401209,False,Other,None,Nallosapare E,19.3600,73.3279,False
4,403-4108307-0229121,2022-04-01,Shipped,Amazon,Expedited,SET044,SET044-KR-NP-L,Set,L,Shipped,...,Hyderabad,Telangana,500024,False,Other,None,Sahifa,17.4171,78.4772,False


In [62]:
# Use city_geo if available, fallback to original city if not
df_amazon_sr['city_final'] = df_amazon_sr['city_geo'].fillna(df_amazon_sr['city'])

In [63]:
df_amazon_sr.drop(columns=['city_geo', 'city_match'], inplace=True)

In [64]:
# Quick check on the structure
df_amazon_sr[['city_final', 'state', 'postal_code', 'latitude', 'longitude']].head()

,city_final,state,postal_code,latitude,longitude
0,Barasat H.O,West Bengal,700124,22.4656,88.7803
1,Chikkabanavara,Karnataka,560090,13.2257,77.5750
2,Basai Road,Haryana,122001,28.4180,76.9839
3,Nallosapare E,Maharashtra,401209,19.3600,73.3279
4,Sahifa,Telangana,500024,17.4171,78.4772


In [65]:
# Drop old city column and replace with trusted GeoNames-based city
df_amazon_sr.drop(columns=['city'], inplace=True)
df_amazon_sr.rename(columns={'city_final': 'city'}, inplace=True)

In [66]:
df_amazon_sr.head()

,order_id,date,status,fulfilment,shipping_method,style,sku,category,size,courier_status,...,currency,amount,state,postal_code,b2b,fulfilled_by,promo_id,latitude,longitude,city
0,404-1445673-1345134,2022-04-01,Shipped - Delivered to Buyer,Merchant,Standard,JNE3724,JNE3724-KR-L,Kurta,L,Shipped,...,INR,495.00,West Bengal,700124,False,Easy Ship,Amazon PLCC Free-Financing,22.4656,88.7803,Barasat H.O
1,406-7387241-2881923,2022-04-01,Shipped,Amazon,Expedited,SET187,SET187-KR-DH-XL,Set,XL,Shipped,...,INR,599.00,Karnataka,560090,False,Other,None,13.2257,77.5750,Chikkabanavara
2,402-9332104-0945115,2022-04-01,Cancelled,Merchant,Standard,JNE3383,JNE3383-KR-A-M,Kurta,M,NaN,...,INR,416.19,Haryana,122001,False,Easy Ship,None,28.4180,76.9839,Basai Road
3,408-1008226-2450750,2022-04-01,Shipped,Amazon,Expedited,JNE3405,JNE3405-KR-L,Kurta,L,Shipped,...,INR,449.00,Maharashtra,401209,False,Other,None,19.3600,73.3279,Nallosapare E
4,403-4108307-0229121,2022-04-01,Shipped,Amazon,Expedited,SET044,SET044-KR-NP-L,Set,L,Shipped,...,INR,542.00,Telangana,500024,False,Other,None,17.4171,78.4772,Sahifa


In [67]:
df_amazon_sr['city'].nunique()

9370

In [68]:
# Check for hidden null values
df_amazon_sr[df_amazon_sr['city'].isin(['', ' '])]

,order_id,date,status,fulfilment,shipping_method,style,sku,category,size,courier_status,...,currency,amount,state,postal_code,b2b,fulfilled_by,promo_id,latitude,longitude,city


In [69]:
# Check for null values in 'city' and 'state' columns
df_amazon_sr[['city', 'state']].isnull().sum()

city     0
state    1
dtype: int64

In [70]:
# Investigate
df_amazon_sr[df_amazon_sr['state'].isnull()]

,order_id,date,status,fulfilment,shipping_method,style,sku,category,size,courier_status,...,currency,amount,state,postal_code,b2b,fulfilled_by,promo_id,latitude,longitude,city
4564,405-0034289-0259545,2022-04-04,Cancelled,Amazon,Expedited,J0077,J0077-SKD-S,Set,S,Cancelled,...,NaN,NaN,None,959121,False,Other,None,NaN,NaN,Apo


In [71]:
# Drop row with 'Apo' city and missing state — cancelled and not monetized
df_amazon_sr = df_amazon_sr[~((df_amazon_sr['city'] == 'Apo') & (df_amazon_sr['state'].isnull()))]

In [72]:
sorted(df_amazon_sr['city'].dropna().unique().tolist())

['(Gandhinagar) Sector 16',
 '(Gandhinagar) Sector 23',
 '(Gandhinagar) Sector 28',
 '(Gandhinagar) Sector 29',
 '(Gandhinagar) Sector 6',
 '12z',
 '14 S Majhiwala',
 '15 Velampalayam',
 '33 Bn. Pac Jhansi',
 '63 Velampalayam',
 '7 GD',
 '71rb',
 '7th Mile Kalimpong',
 'A D Market',
 'A F Station Yelahanka',
 'A K Pora',
 'A Kattupadi',
 'A Kothapalle',
 'A. H. Guard',
 'A. Kiliyanur',
 'A.A. College',
 'A.B.V.Palem',
 'A.D. Project',
 'A.G.College',
 'A.G.Office',
 'A.Gs. Staff Quarters',
 'A.I.E. R.C.puram',
 'A.Katapali',
 'A.P.Colony (Karim Nagar)',
 'A.P.Dabolim',
 'A.Pallipatti',
 'AIIMS',
 'AIMS Ponekkara',
 'Aami',
 'Aamtara',
 'Aanole',
 'Aareymilk Colony',
 'Abadi Ramagundam',
 'Abash',
 'Abbarajupalem',
 'Abdalpur',
 'Abhanga',
 'Abhayagiri',
 'Abhiya Kalan',
 'Abholi',
 'Abhoynagar',
 'Abhva',
 'Abi Karlpora',
 'Abinash Chaowdhury Lane',
 'Abishegapakkam',
 'Abubshahar',
 'Abul Fazal Enclave-I',
 'Abulga BK',
 'Aburoad Gandhi Nagar',
 'Acchad',
 'Acharya Vihar',
 'Achhipur'

In [73]:
# Check column order
df_amazon_sr.columns.tolist()

['order_id',
 'date',
 'status',
 'fulfilment',
 'shipping_method',
 'style',
 'sku',
 'category',
 'size',
 'courier_status',
 'qty',
 'currency',
 'amount',
 'state',
 'postal_code',
 'b2b',
 'fulfilled_by',
 'promo_id',
 'latitude',
 'longitude',
 'city']

In [74]:
# Reorder column names
df_amazon_sr = df_amazon_sr[
    [
        'date', 'order_id', 'shipping_method', 'status', 'fulfilment', 
        'courier_status', 'category', 'style', 'sku', 'size', 'qty',
        'amount', 'currency', 'city','state', 'postal_code',
        'latitude', 'longitude', 'promo_id', 'fulfilled_by', 'b2b'
    ]
]

In [75]:
df_amazon_sr.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,amount,currency,city,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,495.00,INR,Barasat H.O,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,599.00,INR,Chikkabanavara,Karnataka,560090,13.2257,77.5750,None,Other,False
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,NaN,Kurta,JNE3383,JNE3383-KR-A-M,M,...,416.19,INR,Basai Road,Haryana,122001,28.4180,76.9839,None,Easy Ship,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,449.00,INR,Nallosapare E,Maharashtra,401209,19.3600,73.3279,None,Other,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,542.00,INR,Sahifa,Telangana,500024,17.4171,78.4772,None,Other,False


In [76]:
# Check for duplicates
print(df_amazon_sr.duplicated().sum())

6


In [77]:
# Inspect the 6 pairs of duplicates
dupes = df_amazon_sr[df_amazon_sr.duplicated(keep=False)]
display(dupes)

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,amount,currency,city,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b
7090,2022-04-06,408-0373839-4433120,Expedited,Cancelled,Amazon,Cancelled,Kurta,JNE3501,JNE3501-KR-M,M,...,NaN,NaN,Nagarbhavi II Stage,Karnataka,560072,13.2257,77.5750,None,Other,False
7091,2022-04-06,408-0373839-4433120,Expedited,Cancelled,Amazon,Cancelled,Kurta,JNE3501,JNE3501-KR-M,M,...,NaN,NaN,Nagarbhavi II Stage,Karnataka,560072,13.2257,77.5750,None,Other,False
18868,2022-04-13,406-0372545-6086735,Expedited,Cancelled,Amazon,Cancelled,Set,SET197,SET197-KR-NP-L,L,...,NaN,NaN,Lohapool,West Bengal,734008,26.9135,88.3947,None,Other,False
18869,2022-04-13,406-0372545-6086735,Expedited,Cancelled,Amazon,Cancelled,Set,SET197,SET197-KR-NP-L,L,...,NaN,NaN,Lohapool,West Bengal,734008,26.9135,88.3947,None,Other,False
53139,2022-05-04,171-3249942-2207542,Expedited,Shipped,Amazon,Shipped,Set,SET323,SET323-KR-NP-XL,XL,...,939.0,INR,Marunji,Maharashtra,411057,18.6032,73.7626,None,Other,False
53140,2022-05-04,171-3249942-2207542,Expedited,Shipped,Amazon,Shipped,Set,SET323,SET323-KR-NP-XL,XL,...,939.0,INR,Marunji,Maharashtra,411057,18.6032,73.7626,None,Other,False
55135,2022-05-04,405-8669298-3850736,Expedited,Shipped,Amazon,Shipped,Kurta,MEN5025,MEN5025-KR-XXXL,3XL,...,533.0,INR,Hasanpur Bhowapur,Uttar Pradesh,201010,28.7643,77.4856,None,Other,False
55136,2022-05-04,405-8669298-3850736,Expedited,Shipped,Amazon,Shipped,Kurta,MEN5025,MEN5025-KR-XXXL,3XL,...,533.0,INR,Hasanpur Bhowapur,Uttar Pradesh,201010,28.7643,77.4856,None,Other,False
60856,2022-05-08,171-9628368-5329958,Expedited,Cancelled,Amazon,Cancelled,Kurta,J0329,J0329-KR-L,L,...,NaN,NaN,Kaloor,Kerala,682017,9.9670,76.3159,None,Other,False
60899,2022-05-08,171-9628368-5329958,Expedited,Cancelled,Amazon,Cancelled,Kurta,J0329,J0329-KR-L,L,...,NaN,NaN,Kaloor,Kerala,682017,9.9670,76.3159,None,Other,False


In [78]:
# Check that they are identical
dupes.iloc[0].equals(dupes.iloc[1])

True

In [79]:
# Drop them and check
df_amazon_sr.drop_duplicates(inplace=True)
print(df_amazon_sr.duplicated().sum())

0


In [80]:
# Check missing values
missing = df_amazon_sr.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
display(missing)

amount            7665
currency          7665
courier_status    6869
latitude          1172
longitude         1172
dtype: int64

In [81]:
# Check missing lat/lon first
df_amazon_sr[df_amazon_sr['latitude'].isnull()][['postal_code', 'state', 'city']].head(10)

,postal_code,state,city
62,600080,Tamil Nadu,Chennai
93,400705,Maharashtra,Navi Mumbai
624,410209,Maharashtra,Navi Mumbai
702,629603,Tamil Nadu,Nagercoil
707,380025,Gujarat,Ahmedabad
730,302023,Rajasthan,Jaipur
746,462021,Madhya Pradesh,Bhopal
849,500073,Andhra Pradesh,Hyderabad
917,688547,Kerala,Aroor
1110,144013,Punjab,Jalandhar


In [82]:
missing_postals = df_amazon_sr[df_amazon_sr['latitude'].isnull()]['postal_code'].nunique()
print(f'Postal codes with missing lat/lon: {missing_postals}')

Postal codes with missing lat/lon: 222


In [83]:
# Note: Although 1,172 rows are missing lat/lon, they come from only 222 unique postal codes.
# Tableau aggregates by postal code, so these rows collapse into fewer map points.
# In Tableau, only 1 unmapped location appears — acceptable for now.
# Leaving these rows in the dataset in case we want to patch coordinates later.

In [84]:
# Missing amount/currency
# Check to see if the 7665 rows with missing amount values were cancelled and that's why they are missing
df_amazon_sr[df_amazon_sr['amount'].isnull()]['status'].value_counts()

status
Cancelled                       7559
Shipped                           93
Shipped - Delivered to Buyer       8
Shipped - Returned to Seller       3
Pending                            2
Name: count, dtype: int64

In [85]:
# Most (7559) but not all (7665) were cancelled. 106 non-cancelled orders are missing currency/amount

In [86]:
# Add a column to flag rows that have a valid (non-null) 'amount' value to track which orders contain real payment data
df_amazon_sr['has_amount'] = df_amazon_sr['amount'].notna()

In [87]:
# Create a separate dataframe for completed, paid orders
df_amazon_sr_paid = df_amazon_sr[
    (df_amazon_sr['amount'].notna()) &
    (df_amazon_sr['status'] != 'Cancelled')
].copy() # prevent unwanted links to the original dataframe                             

In [88]:
df_amazon_sr.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,currency,city,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,INR,Barasat H.O,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,INR,Chikkabanavara,Karnataka,560090,13.2257,77.5750,None,Other,False,True
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,NaN,Kurta,JNE3383,JNE3383-KR-A-M,M,...,INR,Basai Road,Haryana,122001,28.4180,76.9839,None,Easy Ship,False,True
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,INR,Nallosapare E,Maharashtra,401209,19.3600,73.3279,None,Other,False,True
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,INR,Sahifa,Telangana,500024,17.4171,78.4772,None,Other,False,True


In [89]:
# Fill missing courier_status values
# Check how many missing values exist in 'courier_status'
df_amazon_sr['courier_status'].isnull().sum()

np.int64(6869)

In [90]:
# Check to see if NaN values in courier_status are == status 'Cancelled'
df_amazon_sr[df_amazon_sr['courier_status'].isnull()]['status'].value_counts()

status
Cancelled                       6858
Shipped - Delivered to Buyer       8
Shipped - Returned to Seller       3
Name: count, dtype: int64

In [91]:
# Of the 6869 missing courier_status values, 6858 are cancelled. Remaining 11 are in-between or post-ship states

In [92]:
# Only fill NaNs in courier_status where status is "Cancelled", leave the 11 NaNs
df_amazon_sr.loc[
    (df_amazon_sr['courier_status'].isnull()) & (df_amazon_sr['status'] == 'Cancelled'),
    'courier_status'
] = 'Unshipped'

df_amazon_sr_paid.loc[
    (df_amazon_sr_paid['courier_status'].isnull()) & (df_amazon_sr_paid['status'] == 'Cancelled'),
    'courier_status'
] = 'Unshipped'

In [93]:
# Check for suspicious rows: qty == 0 but status is not cancelled
df_amazon_sr[(df_amazon_sr['qty'] == 0) & (df_amazon_sr['status'] != 'Cancelled')].head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,currency,city,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount
116,2022-04-01,407-9562445-9572337,Expedited,Shipped,Amazon,Cancelled,Kurta,JNE3465,JNE3465-KR-L,L,...,NaN,Karnal Sadar Bazar,Haryana,132001,29.6875,76.8847,None,Other,False,False
506,2022-04-02,407-7550181-9573127,Expedited,Shipped,Amazon,Cancelled,Set,SET331,SET331-KR-NP-XS,XS,...,NaN,Rangailunda,Odisha,760007,19.3355,84.8685,None,Other,False,False
543,2022-04-02,405-7650095-6300317,Expedited,Shipped,Amazon,Cancelled,Kurta,JNE3405,JNE3405-KR-S,S,...,NaN,J.J.Hospital,Maharashtra,400008,18.9474,72.8138,None,Other,False,False
1414,2022-04-02,403-7603946-1101919,Expedited,Shipped,Amazon,Cancelled,Set,J0251,J0251-SKD-XS,XS,...,NaN,Hastal Village,Delhi,110059,28.6553,77.0657,None,Other,False,False
1468,2022-04-02,406-7490066-8351568,Expedited,Shipped,Amazon,Cancelled,Set,SET349,SET349-KR-NP-L,L,...,NaN,Sector 44 (Chandigarh),Chandigarh,160047,30.7080,76.7621,None,Other,False,False


In [94]:
# Drop rows with qty == 0 that aren't cancelled (likely invalid)
df_amazon_sr = df_amazon_sr[~((df_amazon_sr['qty'] == 0) & (df_amazon_sr['status'] != 'Cancelled'))]

In [95]:
# Clean up 'status' colums values

In [96]:
df_amazon_sr['status'].unique()

array(['Shipped - Delivered to Buyer', 'Shipped', 'Cancelled',
       'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
       'Pending', 'Shipped - Picked Up', 'Shipped - Returning to Seller',
       'Shipped - Out for Delivery', 'Shipped - Lost in Transit',
       'Shipped - Damaged', 'Pending - Waiting for Pick Up'], dtype=object)

In [97]:
# Check all unique 'status' values and how many times each appears
df_amazon_sr['status'].value_counts(dropna=False)

status
Shipped                          77577
Shipped - Delivered to Buyer     28754
Cancelled                        18320
Shipped - Returned to Seller      1947
Shipped - Picked Up                973
Pending                            656
Pending - Waiting for Pick Up      281
Shipped - Returning to Seller      145
Shipped - Out for Delivery          35
Shipped - Rejected by Buyer         11
Shipped - Lost in Transit            5
Shipped - Damaged                    1
Name: count, dtype: int64

In [98]:
# Drop 6 rows with problematic shipping statuses — damaged, or lost in transit
problematic_statuses = [
    'Shipped - Damaged',
    'Shipped - Lost in Transit'
]

df_amazon_sr = df_amazon_sr[
    ~df_amazon_sr['status'].isin(problematic_statuses)
]
df_amazon_sr_paid = df_amazon_sr_paid[
    ~df_amazon_sr_paid['status'].isin(problematic_statuses)
]

In [99]:
# Create a simplified 'status_clean' column for grouping and analysis

def clean_order_status(status, amount):
    if status.startswith('Cancelled'):
        return 'Cancelled'
    if status.startswith('Pending'):
        return 'Pending'
    if status in [
        'Shipped',
        'Shipped - Delivered to Buyer',
        'Shipped - Picked Up',
        'Shipped - Out for Delivery'
    ]:
        if amount == 0:
            return 'Shipped - Replacement'
        return 'Shipped'
    if status in [
        'Shipped - Returned to Seller',
        'Shipped - Rejected by Buyer',
        'Shipped - Returning to Seller'
    ]:
        return 'Returned'
    return 'Unknown'

In [100]:
df_amazon_sr['status_clean'] = df_amazon_sr.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

df_amazon_sr_paid['status_clean'] = df_amazon_sr_paid.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

In [101]:
# Cross-tab to check how status_clean aligns with courier_status
df_amazon_sr.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

courier_status,Cancelled,Shipped,Unshipped
status_clean,,,
Cancelled,5834,0,12486
Pending,0,10,927
Returned,0,2103,0
Shipped,0,105077,0
Shipped - Replacement,0,2262,0


In [102]:
# Tag weird rows with new column status_mismatch
df_amazon_sr['status_mismatch'] = (
    ((df_amazon_sr['status_clean'] == 'Pending') & (df_amazon_sr['courier_status'] == 'Shipped'))
)
df_amazon_sr.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Karnataka,560090,13.2257,77.5750,None,Other,False,True,Shipped,False
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,Unshipped,Kurta,JNE3383,JNE3383-KR-A-M,M,...,Haryana,122001,28.4180,76.9839,None,Easy Ship,False,True,Cancelled,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Maharashtra,401209,19.3600,73.3279,None,Other,False,True,Shipped,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Telangana,500024,17.4171,78.4772,None,Other,False,True,Shipped,False


In [103]:
df_amazon_sr_paid.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

courier_status,Shipped,Unshipped
status_clean,,
Pending,10,927
Returned,2103,0
Shipped,105077,0
Shipped - Replacement,2262,0


In [104]:
# Tag weird rows with new column status_mismatch
df_amazon_sr_paid['status_mismatch'] = (
        ((df_amazon_sr_paid['status_clean'] == 'Pending') & (df_amazon_sr_paid['courier_status'] == 'Shipped'))
)
df_amazon_sr_paid.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Karnataka,560090,13.2257,77.5750,None,Other,False,True,Shipped,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Maharashtra,401209,19.3600,73.3279,None,Other,False,True,Shipped,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Telangana,500024,17.4171,78.4772,None,Other,False,True,Shipped,False
8,2022-04-01,406-1810033-9165961,Expedited,Shipped,Amazon,Shipped,Set,J0345,J0345-SET-M,M,...,Karnataka,560085,13.2257,77.5750,None,Other,False,True,Shipped,False


In [105]:
# Check both dataframes: 
# df_amazon_sr should have null values only for currency/amount and lat/lon
# df_amazon_sr_paid should only have null for lat/lon

In [106]:
df_amazon_sr.isnull().sum()

date                  0
order_id              0
shipping_method       0
status                0
fulfilment            0
courier_status        0
category              0
style                 0
sku                   0
size                  0
qty                   0
amount             7559
currency           7559
city                  0
state                 0
postal_code           0
latitude           1172
longitude          1172
promo_id              0
fulfilled_by          0
b2b                   0
has_amount            0
status_clean          0
status_mismatch       0
dtype: int64

In [107]:
df_amazon_sr_paid.isnull().sum()

date                  0
order_id              0
shipping_method       0
status                0
fulfilment            0
courier_status        0
category              0
style                 0
sku                   0
size                  0
qty                   0
amount                0
currency              0
city                  0
state                 0
postal_code           0
latitude           1024
longitude          1024
promo_id              0
fulfilled_by          0
b2b                   0
has_amount            0
status_clean          0
status_mismatch       0
dtype: int64

In [108]:
# Cleaning done! Reset the indexes. 
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr.reset_index(drop=True, inplace=True)
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr_paid.reset_index(drop=True, inplace=True)

In [109]:
# Inspect the final clean versions!

In [110]:
df_amazon_sr.describe()

,date,qty,amount,latitude,longitude
count,128699,128699.000000,121140.000000,127527.000000,127527.000000
mean,2022-05-13 11:52:27.856626432,0.904786,648.573876,19.841107,78.274407
min,2022-04-01 00:00:00,0.000000,0.000000,8.099100,68.757400
25%,2022-04-21 00:00:00,1.000000,449.000000,13.225700,75.763300
50%,2022-05-11 00:00:00,1.000000,605.000000,18.980800,77.575000
75%,2022-06-05 00:00:00,1.000000,788.000000,25.597700,79.810700
max,2022-06-30 00:00:00,8.000000,5584.000000,34.531900,96.195200
std,NaN,0.307121,281.208949,6.228809,4.416596


In [111]:
df_amazon_sr_paid.describe()

,date,qty,amount,latitude,longitude
count,110379,110379.000000,110379.000000,109355.000000,109355.000000
mean,2022-05-13 14:03:15.102329344,1.003733,649.136022,19.847478,78.248877
min,2022-04-01 00:00:00,1.000000,0.000000,8.099100,68.757400
25%,2022-04-21 00:00:00,1.000000,449.000000,13.225700,75.762800
50%,2022-05-11 00:00:00,1.000000,606.000000,18.980800,77.575000
75%,2022-06-05 00:00:00,1.000000,788.000000,25.599000,79.777300
max,2022-06-30 00:00:00,8.000000,5584.000000,34.531900,96.195200
std,NaN,0.072892,283.205182,6.219281,4.410860


In [112]:
df_amazon_sr.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Karnataka,560090,13.2257,77.5750,None,Other,False,True,Shipped,False
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,Unshipped,Kurta,JNE3383,JNE3383-KR-A-M,M,...,Haryana,122001,28.4180,76.9839,None,Easy Ship,False,True,Cancelled,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Maharashtra,401209,19.3600,73.3279,None,Other,False,True,Shipped,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Telangana,500024,17.4171,78.4772,None,Other,False,True,Shipped,False


In [113]:
df_amazon_sr_paid.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Karnataka,560090,13.2257,77.5750,None,Other,False,True,Shipped,False
2,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Maharashtra,401209,19.3600,73.3279,None,Other,False,True,Shipped,False
3,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Telangana,500024,17.4171,78.4772,None,Other,False,True,Shipped,False
4,2022-04-01,406-1810033-9165961,Expedited,Shipped,Amazon,Shipped,Set,J0345,J0345-SET-M,M,...,Karnataka,560085,13.2257,77.5750,None,Other,False,True,Shipped,False


In [114]:
# We will use df_amazon_sr_paid for financial analysis (real, completed sales only)

In [115]:
# For SKU-level or category-level trend analysis, sales volume, promo use, and time-based visualizations,
# we'll create a new version (df_amazon_sr_mean) with missing 'amount' values filled using the mean price per SKU

df_amazon_sr_mean = df_amazon_sr.copy()

# Flag rows where 'amount' was originally missing (for transparency in analysis)
df_amazon_sr_mean['amount_filled'] = df_amazon_sr_mean['amount'].isnull()

# Build a mapping of SKU -> average amount
sku_mean_map = (
    df_amazon_sr_mean.groupby('sku')['amount']
    .mean()
    .to_dict()
)

# Fill missing 'amount' values using the SKU-level mean
df_amazon_sr_mean['amount'] = df_amazon_sr_mean['amount'].fillna(
    df_amazon_sr_mean['sku'].map(sku_mean_map)
)

# Fill any remaining missing currency values with 'INR'
df_amazon_sr_mean['currency'] = df_amazon_sr_mean['currency'].fillna('INR')

In [116]:
# Check null values, should be none (except lat/lon)
df_amazon_sr_mean.isnull().sum()

date                  0
order_id              0
shipping_method       0
status                0
fulfilment            0
courier_status        0
category              0
style                 0
sku                   0
size                  0
qty                   0
amount               34
currency              0
city                  0
state                 0
postal_code           0
latitude           1172
longitude          1172
promo_id              0
fulfilled_by          0
b2b                   0
has_amount            0
status_clean          0
status_mismatch       0
amount_filled         0
dtype: int64

In [117]:
# Inspect the 34 skus with null values for amount
df_amazon_sr_mean[df_amazon_sr_mean['amount'].isnull()]['sku'].value_counts()

sku
BTM002-B-XXL          4
J0184-KR-A-L          2
SET226-KR-PP-L        2
BL100-XXL             2
J0086-DR-XL           1
JNE3557-KR-XL         1
J0024-KR-XXXL         1
J0155-KR-XS           1
J0250-SKD-XS          1
JNE3364-KR-1051-XS    1
SET083-KR-PP-L        1
SET043-KR-NP-XXL      1
JNE2294-KR-A-XS       1
JNE3780-KR-XS         1
BL091-XL              1
JNE3662-TP-M          1
JNE3447-KR-XL         1
J0034-SET-S           1
JNE2145-KR-A-L        1
JNE3483-KR-M          1
JNE3403-KR-XXL        1
SAR182                1
JNE3897-KR-M          1
J0073-KR-S            1
JNE3891-TP-XXL        1
SAR060                1
JNE3909-KR-L          1
J0374-KR-M            1
Name: count, dtype: int64

In [118]:
# Drop 34 remaining rows where 'amount' could not be calculated (no valid price history for the sku)
df_amazon_sr_mean = df_amazon_sr_mean[df_amazon_sr_mean['amount'].notna()]

In [119]:
# Final check
df_amazon_sr_mean.isnull().sum()

date                  0
order_id              0
shipping_method       0
status                0
fulfilment            0
courier_status        0
category              0
style                 0
sku                   0
size                  0
qty                   0
amount                0
currency              0
city                  0
state                 0
postal_code           0
latitude           1171
longitude          1171
promo_id              0
fulfilled_by          0
b2b                   0
has_amount            0
status_clean          0
status_mismatch       0
amount_filled         0
dtype: int64

In [120]:
# Apply same function for the 'status' column to add clean 'order_status' column
df_amazon_sr_mean['status_clean'] = df_amazon_sr_mean.apply(
    lambda row: clean_order_status(row['status'], row['amount']),
    axis=1
)

In [121]:
df_amazon_sr_paid.groupby(['status_clean', 'courier_status']).size().unstack(fill_value=0)

courier_status,Shipped,Unshipped
status_clean,,
Pending,10,927
Returned,2103,0
Shipped,105077,0
Shipped - Replacement,2262,0


In [122]:
# Tag weird rows with new column status_mismatch
df_amazon_sr_mean['status_mismatch'] = (
    ((df_amazon_sr_mean['status_clean'] == 'Pending') & (df_amazon_sr_mean['courier_status'] == 'Shipped'))
)
df_amazon_sr_mean.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch,amount_filled
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,560090,13.2257,77.5750,None,Other,False,True,Shipped,False,False
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,Unshipped,Kurta,JNE3383,JNE3383-KR-A-M,M,...,122001,28.4180,76.9839,None,Easy Ship,False,True,Cancelled,False,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,401209,19.3600,73.3279,None,Other,False,True,Shipped,False,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,500024,17.4171,78.4772,None,Other,False,True,Shipped,False,False


In [123]:
# Reset the index so it starts at 0 and removes the old index completely
df_amazon_sr_mean.reset_index(drop=True, inplace=True)

In [124]:
# Confirm shape for each data frame
print ('df_amazon_sr',df_amazon_sr.shape)
print('df_amazon_sr_paid', df_amazon_sr_paid.shape)
print('df_amazon_sr_mean', df_amazon_sr_mean.shape)

df_amazon_sr (128699, 24)
df_amazon_sr_paid (110379, 24)
df_amazon_sr_mean (128665, 25)


In [125]:
# Dataframes are clean but there are category mismatches with cleaned sale_report.csv

In [126]:
# Load in the dataframe to compare

df_sr = pd.read_csv('../cleaned_data/sale_report.csv')

In [127]:
# Merge on sku to compare category values
df_compare = df_amazon_sr.merge(
    df_sr,
    how='inner',
    on='sku',
    suffixes=('_amazon', '_sr')
)
df_compare.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category_amazon,style,sku,size_amazon,...,fulfilled_by,b2b,has_amount,status_clean,status_mismatch,design_no,stock,category_sr,size_sr,color
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,Easy Ship,False,True,Shipped,False,JNE3724,8,Kurta,L,Pink
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Other,False,True,Shipped,False,SET187,71,Kurta Set,XL,Peach
2,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Other,False,True,Shipped,False,JNE3405,446,Kurta,L,Pink
3,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Other,False,True,Shipped,False,SET044,19,Kurta Set,L,Mustard
4,2022-04-01,407-0339645-8708348,Standard,Cancelled,Merchant,Unshipped,Western Dress,JNE3797,JNE3797-KR-L,L,...,Easy Ship,False,True,Cancelled,False,JNE3797,5,Kurta Set,L,Green


In [128]:
# Keep only rows where the category values don’t match between the two sources
df_disparities = df_compare[
    df_compare['category_amazon'] != df_compare['category_sr']
].copy()

# Drop duplicates based on style and source categories -— no need for every size/color variant
df_disparities_unique = df_disparities.drop_duplicates(
    subset=['style', 'design_no', 'category_amazon', 'category_sr']
)
df_disparities_unique.head()

,date,order_id,shipping_method,status,fulfilment,courier_status,category_amazon,style,sku,size_amazon,...,fulfilled_by,b2b,has_amount,status_clean,status_mismatch,design_no,stock,category_sr,size_sr,color
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Set,SET187,SET187-KR-DH-XL,XL,...,Other,False,True,Shipped,False,SET187,71,Kurta Set,XL,Peach
3,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Set,SET044,SET044-KR-NP-L,L,...,Other,False,True,Shipped,False,SET044,19,Kurta Set,L,Mustard
4,2022-04-01,407-0339645-8708348,Standard,Cancelled,Merchant,Unshipped,Western Dress,JNE3797,JNE3797-KR-L,L,...,Easy Ship,False,True,Cancelled,False,JNE3797,5,Kurta Set,L,Green
5,2022-04-01,171-1556000-8993931,Expedited,Cancelled,Amazon,Cancelled,Set,SET282,SET282-KR-PP-XXXL,3XL,...,Other,False,False,Cancelled,False,SET282,0,Kurta Set,3XL,Yellow
6,2022-04-01,403-9536559-5724368,Expedited,Cancelled,Amazon,Cancelled,Set,SET327,SET327-KR-DPT-L,L,...,Other,False,False,Cancelled,False,SET327,12,Kurta Set,L,Black


In [129]:
# Pull just the columns I want/need to review and correct — cleaner to work with in excel
df_export = df_disparities_unique[[
    'sku', 'style', 'category_amazon', 'design_no', 'category_sr'
]]

In [130]:
# Make a safe copy so I can add a column (without pandas throwing warnings)
df_export = df_export.copy()
df_export['correct_category'] = ''

In [131]:
# Save to csv to work on in excel
df_export.to_csv('../data/category_disparities.csv', index=False)

In [132]:
# Import csv with category_fixed filled
df_fixes = pd.read_csv('../data/category_disparities_fixed.csv')

In [133]:
# Create a style -> correct_category map
fix_map = dict(zip(df_fixes['style'], df_fixes['correct_category']))

In [134]:
# Update category only where I’ve provided a correction — keep the original if it’s not in the fix map
df_amazon_sr['category'] = df_amazon_sr.apply(
    lambda row: fix_map[row['style']] if row['style'] in fix_map else row['category'],
    axis=1
)

In [135]:
# Check - as expected, can see both updated styles and old ones
df_amazon_sr.head(10)

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
0,2022-04-01,404-1445673-1345134,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Kurta,JNE3724,JNE3724-KR-L,L,...,West Bengal,700124,22.4656,88.7803,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
1,2022-04-01,406-7387241-2881923,Expedited,Shipped,Amazon,Shipped,Kurta Set,SET187,SET187-KR-DH-XL,XL,...,Karnataka,560090,13.2257,77.5750,None,Other,False,True,Shipped,False
2,2022-04-01,402-9332104-0945115,Standard,Cancelled,Merchant,Unshipped,Kurta,JNE3383,JNE3383-KR-A-M,M,...,Haryana,122001,28.4180,76.9839,None,Easy Ship,False,True,Cancelled,False
3,2022-04-01,408-1008226-2450750,Expedited,Shipped,Amazon,Shipped,Kurta,JNE3405,JNE3405-KR-L,L,...,Maharashtra,401209,19.3600,73.3279,None,Other,False,True,Shipped,False
4,2022-04-01,403-4108307-0229121,Expedited,Shipped,Amazon,Shipped,Kurta Set,SET044,SET044-KR-NP-L,L,...,Telangana,500024,17.4171,78.4772,None,Other,False,True,Shipped,False
5,2022-04-01,407-0339645-8708348,Standard,Cancelled,Merchant,Unshipped,Kurta Set,JNE3797,JNE3797-KR-L,L,...,Kerala,673101,11.6103,75.7806,None,Easy Ship,False,True,Cancelled,False
6,2022-04-01,171-1556000-8993931,Expedited,Cancelled,Amazon,Cancelled,Kurta Set,SET282,SET282-KR-PP-XXXL,3XL,...,West Bengal,700029,22.5553,88.3558,None,Other,False,False,Cancelled,False
7,2022-04-01,403-9536559-5724368,Expedited,Cancelled,Amazon,Cancelled,Set,SET327,SET327-KR-DPT-L,L,...,Telangana,500020,17.4057,78.4944,None,Other,False,False,Cancelled,False
8,2022-04-01,406-1810033-9165961,Expedited,Shipped,Amazon,Shipped,Set,J0345,J0345-SET-M,M,...,Karnataka,560085,13.2257,77.5750,None,Other,False,True,Shipped,False
9,2022-04-01,407-3386003-0476346,Expedited,Shipped,Amazon,Shipped,Set,SET366,SET366-KR-NP-S,S,...,Maharashtra,421302,19.2166,73.2176,None,Other,False,True,Shipped,False


In [136]:
# Check out the value counts
df_amazon_sr['category'].value_counts()

category
Kurta                    49754
Kurta Set                38645
Set                      14309
Western Dress             9385
Top                       8936
Night Wear                2313
Tunic                     1453
Ethnic Dress              1158
Blouse                     922
Crop Top With Palazzo      498
Lehenga Choli              484
Crop Top                   201
Palazzo                    187
Saree                      164
Leggings                   124
Pant                       122
Cardigan                    20
Jumpsuit                    15
Bottom                       6
Dupatta                      3
Name: count, dtype: int64

In [137]:
# Categories are cleaner. The remaing Sets are likely Kurta Set; 
# Tops are either Tunic, Blouse, or Crop Top, 
# Bottoms are either Palazzo, Leggings, or Pants

In [138]:
# The sku for Kurta Sets have either 'KR', 'SET' or 'SKD' 
# We should catch them all here:

mask_set_kr_set_skd = (
    (df_amazon_sr['category'] == 'Set') &
    (df_amazon_sr['sku'].str.contains('KR|SET|SKD', na=False))
)
df_amazon_sr.loc[mask_set_kr_set_skd, 'category'] = 'Kurta Set'

In [139]:
# Check - got them all!
df_amazon_sr['category'].value_counts()

category
Kurta Set                52954
Kurta                    49754
Western Dress             9385
Top                       8936
Night Wear                2313
Tunic                     1453
Ethnic Dress              1158
Blouse                     922
Crop Top With Palazzo      498
Lehenga Choli              484
Crop Top                   201
Palazzo                    187
Saree                      164
Leggings                   124
Pant                       122
Cardigan                    20
Jumpsuit                    15
Bottom                       6
Dupatta                      3
Name: count, dtype: int64

In [140]:
# The sku for Tunic contains TU
# The sku for Blouse contains BL
# The sku for Crop Top contains TP
# The sku for Cardigan contains CD
# Let's filter those Tops

mask_top_tu = (
    (df_amazon_sr['category'] == 'Top') &
    (df_amazon_sr['sku'].str.contains('TU', na=False))
)
df_amazon_sr.loc[mask_top_tu, 'category'] = 'Tunic'

mask_top_bl = (
    (df_amazon_sr['category'] == 'Top') &
    (df_amazon_sr['sku'].str.contains('BL', na=False))
)
df_amazon_sr.loc[mask_top_bl, 'category'] = 'Blouse'

mask_top_tp = (
    (df_amazon_sr['category'] == 'Top') &
    (df_amazon_sr['sku'].str.contains('TP', na=False))
)
df_amazon_sr.loc[mask_top_tp, 'category'] = 'Crop Top'

mask_top_cd = (
    (df_amazon_sr['category'] == 'Top') &
    (df_amazon_sr['sku'].str.contains('CD', na=False))
)
df_amazon_sr.loc[mask_top_cd, 'category'] = 'Cardigan'

In [141]:
df_amazon_sr['category'].value_counts()
# Got them all again :)

category
Kurta Set                52954
Kurta                    49754
Western Dress             9385
Crop Top                  8989
Night Wear                2313
Tunic                     1600
Ethnic Dress              1158
Blouse                     922
Crop Top With Palazzo      498
Lehenga Choli              484
Palazzo                    187
Saree                      164
Leggings                   124
Pant                       122
Cardigan                    21
Jumpsuit                    15
Bottom                       6
Dupatta                      3
Name: count, dtype: int64

In [142]:
# Since there are only 6 Bottoms, let's have a look
df_amazon_sr[df_amazon_sr['category'] == 'Bottom']

,date,order_id,shipping_method,status,fulfilment,courier_status,category,style,sku,size,...,state,postal_code,latitude,longitude,promo_id,fulfilled_by,b2b,has_amount,status_clean,status_mismatch
32667,2022-04-21,403-6781808-5068367,Expedited,Shipped,Amazon,Shipped,Bottom,BTM021,BTM021-B-XS,XS,...,West Bengal,711106,22.5260,88.0676,None,Other,False,True,Shipped,False
33870,2022-04-22,406-2345701-7973952,Expedited,Shipped,Amazon,Shipped,Bottom,BTM021,BTM021-B-L,L,...,Maharashtra,400050,18.9808,72.8338,None,Other,False,True,Shipped,False
39527,2022-04-25,408-6022939-2897110,Standard,Shipped,Amazon,Shipped,Bottom,BTM021,BTM021-B-XS,XS,...,West Bengal,711106,22.5260,88.0676,None,Other,False,True,Shipped - Replacement,False
41874,2022-04-27,407-4754304-8740360,Expedited,Shipped,Amazon,Shipped,Bottom,BTM021,BTM021-B-XXL,XXL,...,Goa,403508,15.5945,73.8682,None,Other,False,True,Shipped,False
79455,2022-05-23,408-4686308-6807529,Standard,Shipped - Delivered to Buyer,Merchant,Shipped,Bottom,BTM004,BTM004-XXXL,3XL,...,Delhi,110005,28.6551,77.1833,Amazon PLCC Free-Financing,Easy Ship,False,True,Shipped,False
86896,2022-05-29,403-5414902-6244310,Expedited,Shipped,Amazon,Shipped,Bottom,BTM021,BTM021-B-XXXL,3XL,...,Tamil Nadu,600019,13.1582,80.3018,None,Other,False,True,Shipped,False


In [143]:
# The sku for Palazzo contains BTM + PP
# The sku for Leggings contains AN + a color
# The sku for Pants contains BTM + NP OR just BTM

# So the remaining bottoms are most likely pants

mask_bottom_btm = (
    (df_amazon_sr['category'] == 'Bottom') &
    (df_amazon_sr['sku'].str.contains('BTM', na=False))
)
df_amazon_sr.loc[mask_bottom_btm, 'category'] = 'Pant'

In [144]:
# Final check - we are clean
df_amazon_sr['category'].value_counts()

category
Kurta Set                52954
Kurta                    49754
Western Dress             9385
Crop Top                  8989
Night Wear                2313
Tunic                     1600
Ethnic Dress              1158
Blouse                     922
Crop Top With Palazzo      498
Lehenga Choli              484
Palazzo                    187
Saree                      164
Pant                       128
Leggings                   124
Cardigan                    21
Jumpsuit                    15
Dupatta                      3
Name: count, dtype: int64

In [145]:
# Build a fresh style -> category map from the cleaned df_amazon_sr
category_map = dict(zip(df_amazon_sr['style'], df_amazon_sr['category']))

In [146]:
# Use that to update categories in _paid & _mean dfs (fall back to existing category if something’s missing)
df_amazon_sr_paid['category'] = df_amazon_sr_paid.apply(
    lambda row: category_map.get(row['style'], row['category']),
    axis=1
)

df_amazon_sr_mean['category'] = df_amazon_sr_mean.apply(
    lambda row: category_map.get(row['style'], row['category']),
    axis=1
)

In [147]:
# Update df_sr using the cleaned categories from df_amazon_sr
# It helped break down 'Dress' into more specific types, and also fixed small stuff like typos and mislabels

category_fix_map = dict(zip(df_amazon_sr['style'], df_amazon_sr['category']))

In [148]:
df_sr['category'] = df_sr.apply(
    lambda row: category_fix_map[row['design_no']]
    if row['design_no'] in category_fix_map
    else row['category'],
    axis=1
)

In [149]:
# # Save the clean dataframes as new .csv files

# df_amazon_sr.to_csv('../cleaned_data/amazon_sales_full.csv', index=False)
# df_amazon_sr_paid.to_csv('../cleaned_data/amazon_sales_paid.csv', index=False)
# df_amazon_sr_mean.to_csv('../cleaned_data/amazon_sales_mean.csv', index=False)

# # Update sale_report.csv

# df_sr.to_csv('../cleaned_data/sale_report.csv', index=False)

# # index=False keeps the row numbers out of the file for cleaner for loading later

In [150]:
# This notebook cleaned the raw Amazon Sales Report CSV and produced three dataframes:
# 1. df_amazon_sr: full cleaned dataset (some included missing 'amount')
# 2. df_amazon_sr_paid: contains only valid, completed sales (non-cancelled + 'amount' present (use: financial analysis)
# 3. df_amazon_sr_mean: includes rows with filled 'amount'  using mean per sku (use: trend analysis)

# This notebook also updated df_sr to match categories